# NOTEBOOK RESUMIDO DE: Javi_Ideas.ipynb

In [ ]:
# descargar librerias si es necesario
!pip install pandas
!pip install xlrd

In [3]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import dayofweek, dayofmonth, weekofyear, month, year

import matplotlib.pyplot as plt
import pandas as pd




#####################################        FUNCIONES BÁSICAS        ########################################


## CARGA DE FICHEROS

def load_df(spark, e): # e € { 2, 3, 4 }
    data_path = f"datos/sample_10e{e}.json"
    return spark.read.json(data_path)

def load_geo():
    return pd.read_excel('datos/bases_bicimad.xls')


## IDEA 1

def demanda_por_zonas(df, plot=False):
    
    df1 = df.groupBy('idunplug_station').count()
    df2 = df.groupBy('idplug_station').count()
    
    # renombrar las columnas para unirlas
    df1 = df1.withColumnRenamed('idunplug_station', 'id').withColumnRenamed('count', 'n_unplugs')
    df2 = df2.withColumnRenamed('idplug_station', 'id').withColumnRenamed('count', 'n_plugs')
    
    df3 = df1.join(df2, on='id')
    
    if plot:
        
        df3.limit(10).toPandas().plot.bar(
            x='id', 
            y=['n_unplugs','n_plugs'], 
            color=['red','green'],
            title="Visualización de la demanda de las estaciones"
        )
    
    return df3


## IDEA 2

def uso_por_edades(df, plot=False):
    
    # crear un df con el timepo total / medio de uso de la bici, por edades
    tiempo_total = df.groupBy('ageRange')\
                                    .sum('travel_time')\
                                    .withColumnRenamed("sum(travel_time)", "total")\
                                    .orderBy('ageRange')

    tiempo_medio = df.groupBy('ageRange')\
                                    .mean('travel_time')\
                                    .withColumnRenamed("avg(travel_time)", "mean")\
                                    .orderBy('ageRange')
    
    # juntar ambos df en uno solo (on="ageRange" para que se junte según las edades)
    tiempos = tiempo_total.join(tiempo_medio, on="ageRange").orderBy("ageRange")
    
    # plotear resultados
    if plot:
        
        tiempos.toPandas().plot.bar(x="ageRange", y="total", color="blue", title="Tiempo total por rango de edades")
        tiempos.toPandas().plot.bar(x="ageRange", y="mean", color="gray", title="Tiempo medio por rango de edades")

    return tiempos

## IDEA 3

def preprocesar_fechas(df):
    df2 = df.withColumn("dayofweek", dayofweek(df.unplug_hourTime.getItem("$date")))\
            .withColumn("weekofyear", weekofyear(df.unplug_hourTime.getItem("$date")))\
            .withColumn("year", year(df.unplug_hourTime.getItem("$date")))\
            .drop('unplug_hourTime')
    return df2


######################################################################################################




spark = SparkSession.builder.getOrCreate()

data_path = "datos/sample_10e3.json"
df = spark.read.json(data_path)

23/05/25 11:29:01 WARN Utils: Your hostname, pclab032 resolves to a loopback address: 127.0.1.1; using 147.96.132.249 instead (on interface eno1)
23/05/25 11:29:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/25 11:29:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/25 11:29:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
